In [1]:
import cloudpickle
import os

In [2]:
os.listdir()

['data',
 'load_model.ipynb',
 'Main.py',
 'model',
 'model.ipynb',
 'style.css',
 'test.py']

In [3]:
# import cloudpickle

# # Load the model
# model_path = "./model/final_model.joblib"
# with open(model_path, 'rb') as f:
#     model = cloudpickle.load(f)

# # Check if the model is a DecisionTreeClassifier or DecisionTreeRegressor
# if hasattr(model, 'tree_'):
#     # Print tree parameters
#     print("Criterion:", model.criterion)
#     print("Max Depth:", model.max_depth)
#     print("Min Samples Split:", model.min_samples_split)
#     print("Min Samples Leaf:", model.min_samples_leaf)
#     print("Max Features:", model.max_features)
#     print("Splitter:", model.splitter)
    
#     # Print feature importances
#     print("Feature Importances:", model.feature_importances_)
    
#     # Print classes
#     print("Classes:", model.classes_)
    
#     # Print tree structure details (example for classification tree)
#     print("Number of nodes:", model.tree_.node_count)
#     print("Tree Depth:", model.tree_.max_depth)
# else:
#     print("The loaded model does not appear to be a DecisionTree model.")


In [4]:
# import pickle
# model_path = "./model/decision_tree_model.pkl"
# with open(model_path, 'rb') as f:
#     model = pickle.load(f)

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import joblib
# Set global random seed
import random
np.random.seed(42)
random.seed(42)

In [6]:
features_to_keep_005 = ['NutritionIntake', 'Cortisol', 'ProBNP', 'UrineOutput', 'K', 'OxygenTherapy', 'BMI', 'Pulse', 'eGFR', 'TSH', 'FT3', 'Cl', 'Na', 'FT4', 'CRP', 'Ca', 'Mg', 'Ure', 'PCT', 'Lactat', 'Antibiotics', 'Adrenalin', 'Noradrenalin', 'Creatinin', 'PaCO2', 'TroponinI']

In [7]:
features_to_keep_0051 = ['NutritionIntake', 'Cortisol', 'ProBNP', 'UrineOutput', 'K', 'OxygenTherapy', 'BMI', 'Pulse', 'eGFR', 'TSH', 'FT3', 'Cl', 'Na', 'FT4', 'CRP', 'Ca', 'Mg', 'Ure', 'PCT', 'Lactat', 'Antibiotics', 'Adrenalin', 'Noradrenalin', 'Creatinin', 'PaCO2', 'TroponinI','Survival']

In [8]:
data = pd.read_excel("./data/BME10.xlsx",engine="openpyxl")

In [9]:
data.columns

Index(['Unnamed: 0', 'PatientID', 'Age', 'Gender', 'BMI',
       'MedicalConditions_InfectiousStatus', 'SurgicalConditions', 'UMC',
       'Pulse_admission', 'RespiratoryRate_admission',
       'BodyTemperature_admission', 'BloodPressure_admission',
       'Spo2_admission', 'Glasgow_admission', 'WBC_admission', 'HCT_admission',
       'PLT_admission', 'pH_admission', 'PaO2_admission', 'PaCO2_admission',
       'HCO3_admission', 'TroponinI_admission', 'CK-MB_admission',
       'ProBNP_admission', 'AST_admission', 'ALT_admission', 'LDH_admission',
       'Albumin_admission', 'BilirubinTP_admission', 'BilirubinTT_admission',
       'BilirubinGT_admission', 'PT_admission', 'aPTT_admission',
       'DDimer_admission', 'Ure_admission', 'Creatinin_admission',
       'eGFR_admission', 'Cortisol_admission', 'TSH_admission',
       'FT3_admission', 'FT4_admission', 'Na_admission', 'K_admission',
       'Cl_admission', 'Ca_admission', 'Mg_admission', 'CRP_admission',
       'PCT_admission', 'Lact

In [10]:
data.dtypes

Unnamed: 0                              int64
PatientID                               int64
Age                                     int64
Gender                                 object
BMI                                   float64
MedicalConditions_InfectiousStatus     object
SurgicalConditions                     object
UMC                                      bool
Pulse_admission                        object
RespiratoryRate_admission              object
BodyTemperature_admission              object
BloodPressure_admission                object
Spo2_admission                         object
Glasgow_admission                      object
WBC_admission                          object
HCT_admission                          object
PLT_admission                          object
pH_admission                           object
PaO2_admission                         object
PaCO2_admission                        object
HCO3_admission                         object
TroponinI_admission               

In [11]:
def remove_suffix(df: pd.DataFrame, suffix: str) -> pd.DataFrame:
    """
    Rename columns by removing a specified suffix from the column names.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    suffix (str): The suffix to remove from the column names.

    Returns:
    pd.DataFrame: DataFrame with renamed columns.
    """
    new_columns = {}

    for col in df.columns:
        if col.endswith(suffix):
            new_col = col[:-len(suffix)]
            new_columns[col] = new_col
        else:
            new_columns[col] = col

    df.rename(columns=new_columns, inplace=True)

    return df

data = remove_suffix(data, '_admission')

In [12]:
data = data[features_to_keep_0051]

In [13]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

# Identify numeric columns
numeric_cols = data.select_dtypes(include=['int64', 'float64']).columns

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Apply MinMaxScaler to numeric columns
data[numeric_cols] = scaler.fit_transform(data[numeric_cols])

# Identify categorical columns
categorical_cols = data.select_dtypes(include=['object']).columns

# Dictionary to store fitted LabelEncoders
label_encoders = {}

# Step 1: Fit and save a LabelEncoder for each categorical column
for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

print("Encoded Training Data:")
print(data)

Encoded Training Data:
    NutritionIntake  Cortisol  ProBNP  UrineOutput  K  OxygenTherapy  \
0                 1         3       2     0.030700  2              3   
1                 1         3       0     1.000000  0              1   
2                 0         3       2     0.320542  2              2   
3                 2         1       0     0.275395  1              5   
4                 1         3       2     0.139955  2              0   
5                 1         3       0     0.162528  2              2   
6                 1         3       2     0.029345  3              1   
7                 1         3       1     0.124153  2              0   
8                 1         3       2     0.162528  2              1   
9                 1         0       0     0.388262  2              2   
10                2         3       2     0.162528  2              5   
11                1         3       0     0.713318  3              1   
12                2         3       2    

In [14]:
data.head()

,NutritionIntake,Cortisol,ProBNP,UrineOutput,K,OxygenTherapy,BMI,Pulse,eGFR,TSH,...,Ure,PCT,Lactat,Antibiotics,Adrenalin,Noradrenalin,Creatinin,PaCO2,TroponinI,Survival
0,1,3,2,0.030700,2,3,0.136902,0,1,3,...,0,2,1,True,1,1,0,3,0,1
1,1,3,0,1.000000,0,1,0.355063,0,2,3,...,2,2,0,False,1,0,2,1,1,0
2,0,3,2,0.320542,2,2,0.248793,1,1,3,...,0,1,1,False,1,1,0,2,1,1
3,2,1,0,0.275395,1,5,0.452581,1,1,3,...,2,0,0,True,1,0,0,1,0,0
4,1,3,2,0.139955,2,0,0.136902,0,2,3,...,1,2,2,True,1,1,0,0,0,1


In [15]:
data.dtypes

NutritionIntake      int32
Cortisol             int32
ProBNP               int32
UrineOutput        float64
K                    int32
OxygenTherapy        int32
BMI                float64
Pulse                int32
eGFR                 int32
TSH                  int32
FT3                  int32
Cl                   int32
Na                   int32
FT4                  int32
CRP                  int32
Ca                   int32
Mg                   int32
Ure                  int32
PCT                  int32
Lactat               int32
Antibiotics           bool
Adrenalin            int32
Noradrenalin         int32
Creatinin            int32
PaCO2                int32
TroponinI            int32
Survival             int32
dtype: object

In [16]:
joblib.dump(scaler, './model/scaler.pkl')

# Save the entire dictionary of LabelEncoders
joblib.dump(label_encoders, './model/label_encoders.pkl')

['label_encoders.pkl']

In [17]:
# joblib.dump(scaler, './model/scaler.joblib')

# # Lưu LabelEncoders
# joblib.dump(label_encoder, './model/label_encoder.joblib')

In [18]:
data.head()

,NutritionIntake,Cortisol,ProBNP,UrineOutput,K,OxygenTherapy,BMI,Pulse,eGFR,TSH,...,Ure,PCT,Lactat,Antibiotics,Adrenalin,Noradrenalin,Creatinin,PaCO2,TroponinI,Survival
0,1,3,2,0.030700,2,3,0.136902,0,1,3,...,0,2,1,True,1,1,0,3,0,1
1,1,3,0,1.000000,0,1,0.355063,0,2,3,...,2,2,0,False,1,0,2,1,1,0
2,0,3,2,0.320542,2,2,0.248793,1,1,3,...,0,1,1,False,1,1,0,2,1,1
3,2,1,0,0.275395,1,5,0.452581,1,1,3,...,2,0,0,True,1,0,0,1,0,0
4,1,3,2,0.139955,2,0,0.136902,0,2,3,...,1,2,2,True,1,1,0,0,0,1


In [19]:
model = DecisionTreeClassifier(random_state=42)
print("Total feature to keep: ", len(features_to_keep_005))
print(features_to_keep_005)
# Split data into features and target
X = data[features_to_keep_005]
y = data['Survival']
groups = np.arange(len(data))
# Initialize Leave-One-Group-Out cross-validation
logo = LeaveOneGroupOut()

# Initialize lists to store metrics
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

# Perform LOGO cross-validation
for train_index, test_index in logo.split(X, y, groups):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            
    # Train the initial model using all features
    model.fit(X_train, y_train)
    
    # Predict and calculate metrics
    y_pred = model.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    precision_scores.append(precision_score(y_test, y_pred, average='macro', zero_division=0))
    recall_scores.append(recall_score(y_test, y_pred, average='macro', zero_division=0))
    f1_scores.append(f1_score(y_test, y_pred, average='macro', zero_division=0))

print("Accuracy", np.sum(accuracy_scores)/len(accuracy_scores))
print("Precision", np.sum(precision_scores)/len(precision_scores))
print("Recall", np.sum(recall_scores)/len(recall_scores))
print("F1-score", np.sum(f1_scores)/len(f1_scores))

Total feature to keep:  26
['NutritionIntake', 'Cortisol', 'ProBNP', 'UrineOutput', 'K', 'OxygenTherapy', 'BMI', 'Pulse', 'eGFR', 'TSH', 'FT3', 'Cl', 'Na', 'FT4', 'CRP', 'Ca', 'Mg', 'Ure', 'PCT', 'Lactat', 'Antibiotics', 'Adrenalin', 'Noradrenalin', 'Creatinin', 'PaCO2', 'TroponinI']
Accuracy 0.9074074074074074
Precision 0.9074074074074074
Recall 0.9074074074074074
F1-score 0.9074074074074074


In [20]:
print(model)

DecisionTreeClassifier(random_state=42)


In [21]:
import joblib
joblib.dump(model,'./model/decision_tree_model.pkl')

['./model/decision_tree_model.pkl']

In [22]:
print(X.dtypes)

NutritionIntake      int32
Cortisol             int32
ProBNP               int32
UrineOutput        float64
K                    int32
OxygenTherapy        int32
BMI                float64
Pulse                int32
eGFR                 int32
TSH                  int32
FT3                  int32
Cl                   int32
Na                   int32
FT4                  int32
CRP                  int32
Ca                   int32
Mg                   int32
Ure                  int32
PCT                  int32
Lactat               int32
Antibiotics           bool
Adrenalin            int32
Noradrenalin         int32
Creatinin            int32
PaCO2                int32
TroponinI            int32
dtype: object


In [23]:
numeric_cols = ['UrineOutput', 'BMI', 'Pulse', 'eGFR', 'FT3', 'Cl', 'Na', 'CRP', 'Ca', 'Mg', 'Ure', 'PCT', 'Lactat', 'Adrenalin', 'Noradrenalin', 'Creatinin', 'PaCO2', 'TroponinI']
categorical_cols = ['NutritionIntake', 'Cortisol', 'ProBNP', 'K', 'OxygenTherapy', 'TSH', 'FT4', 'Antibiotics']

In [24]:
features_to_keep_0051 = ['NutritionIntake', 'Cortisol', 'ProBNP', 'UrineOutput', 'K', 'OxygenTherapy', 
                        'BMI', 'Pulse', 'eGFR', 'TSH', 'FT3', 'Cl', 'Na', 'FT4', 'CRP', 
                        'Ca', 'Mg', 'Ure', 'PCT', 'Lactat', 'Antibiotics', 'Adrenalin', 
                        'Noradrenalin', 'Creatinin', 'PaCO2', 'TroponinI']

In [25]:
numeric_cols = ['UrineOutput', 'BMI', 'Pulse', 'eGFR', 'FT3', 'Cl', 'Na', 'CRP', 'Ca', 'Mg', 'Ure', 'PCT', 'Lactat', 'Adrenalin', 'Noradrenalin', 'Creatinin', 'PaCO2', 'TroponinI']
categorical_cols = ['NutritionIntake', 'Cortisol', 'ProBNP', 'K', 'OxygenTherapy', 'TSH', 'FT4', 'Antibiotics']

# Ensure all columns in features_to_keep_005 are accounted for
all_features = set(features_to_keep_0051)
numeric_cols = set(numeric_cols)
categorical_cols = set(categorical_cols)

missing_from_categorical = all_features - numeric_cols - categorical_cols
missing_from_numeric = all_features - categorical_cols - numeric_cols

print("Missing from categorical_cols:", missing_from_categorical)
print("Missing from numeric_cols:", missing_from_numeric)

Missing from categorical_cols: set()
Missing from numeric_cols: set()
